In [1]:
import pandas as pd
import numpy as np
import pickle

import math
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
# Set font file path
font_path = '/data/wangyuhang/ttf/Times New Roman.ttf'
# font = FontProperties(fname=font_path, size=12)

from collections import Counter
from itertools import chain
from scipy.optimize import curve_fit
import powerlaw

from rapidfuzz import process, fuzz
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('/data/huggingface/hub/sentence-transformers_all-MiniLM-L6-v2')
import oracledb
connection = oracledb.connect(user="", password="", host="",port=,service_name="")

import oracledb
from sqlalchemy import create_engine
import cx_Oracle
from tqdm import tqdm


from sentence_transformers.util import cos_sim
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.font_manager import FontProperties
from powerlaw import Fit, plot_pdf

from tqdm import tqdm
import re
import os

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, roc_curve,precision_recall_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
# 下载NLTK的punkt数据
nltk.download('punkt')
stemmer = PorterStemmer()

def get_dict(x,dictionary):
    return [dictionary[str(item)] for item in x]
def get_larger(x,number):
    return [item for item in x if item>number]
def compare_larger(x,number):
    return any([item>number for item in x])
def convert_date(x):
    try:
        date=pd.Timestamp(x)
    except:
        date=pd.Timestamp(x.split('-')[0]+'-01-01')
    return date
from aquarel import load_theme
theme = load_theme("arctic_light") #调用aquarel中的'arctic_dark',"arctic_light"

[nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
[nltk_data]     refused>


In [2]:
df_tls901_filting=pd.read_csv('data/tls901_filting.csv')
df_tls901_filting=df_tls901_filting.drop_duplicates()
list_tls901_filting_appln_id=df_tls901_filting['appln_id'].astype(str).unique()
df_tls901_filting

,appln_filing_year,appln_auth,appln_id
0,2007,EP,144
3,2008,EP,217
4,2001,EP,16
9,2002,EP,25
23,2002,EP,32
...,...,...,...
39416470,2020,CN,556845644
39416471,2020,KR,560077487
39416475,2020,TW,562113122
39416476,2021,DE,566482136


In [3]:
# df_scopus = pd.read_sql(sql='SELECT t."eid", t."doi", t."title", t."coverDate", t."afid", t."affilname",\
#         t."affiliation_city", t."affiliation_country", t."author_count",\
#         t."author_names", t."author_ids", t."author_afids",  t."citedby_count",\
#         t."openaccess", t."freetoread", t."freetoreadLabel", t."fund_acr", t."fund_no",\
#         t."fund_sponsor" FROM "biotechnology_paper_basic_info" t', con=connection)
df_scopus=pd.read_pickle('data/biotechnology_paper_basic_info.pickle')
df_scopus['coverDate']=df_scopus['coverDate'].apply(lambda x:convert_date(x))
df_scopus['citedby_count']=df_scopus['citedby_count'].astype(int)
df_scopus

,eid,doi,title,coverDate,afid,affilname,affiliation_city,affiliation_country,author_count,author_names,author_ids,author_afids,citedby_count,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor
0,2-s2.0-84991318817,10.1007/s13205-015-0303-8,Screening and structure-based modeling of T-ce...,2015-12-01,60097662;60094550;60009689,"JH Government Post Graduate College, Betul;Man...",Betul;Aligarh;Gwalior,India;India;India,2.0,"Kamthania, Mohit;Sharma, D. K.",55479937900;57206773481,60094550-60009689;60097662,16,1,repositoryvor,Green,None,undefined,None
1,2-s2.0-84982787473,10.1007/s13205-015-0296-3,Antioxidant activity and structural features o...,2015-12-01,60099701;60095158;60030482;100318031,Acharya Brojendra Nath Seal College;Hooghly Mo...,Cooch Behar;Hooghly;Bardhaman;Durgapur,India;India;India;India,5.0,"Ghosh, Tuhin;Basu, Ankita;Adhikari, Dipan;Roy,...",57197638841;7402167211;57213724746;57193908340...,100318031;60030482;60095158;60099701;100318031,20,1,repositoryam,Green,UGC,F.PSW-70/12-13(ERO),University Grants Commission
2,2-s2.0-84988386029,10.1007/s13205-014-0272-3,Eco-friendly and green synthesis of silver nan...,2015-10-22,60023968,Kakatiya University,Warangal,India,3.0,"Kagithoju, Srikanth;Godishala, Vikram;Nanna, R...",55584286700;55584120800;39762350700,60023968;60023968;60023968,36,1,repositoryvor,Green,None,undefined,None
3,2-s2.0-84983779033,10.1007/s13205-014-0271-4,An alternative method for production of microa...,2015-10-22,60027171,Annamalai University,Chidambaram,India,3.0,"Surendhiran, Duraiarasan;Sirajunnisa, Abdul Ra...",55735347700;56034893200;24468826200,60027171;60027171;60027171,22,1,repositoryvor,Green,None,undefined,None
4,2-s2.0-85016090291,10.1007/s13205-014-0231-z,Structure-based functional annotation of putat...,2015-06-22,60020458,Jamia Millia Islamia,New Delhi,India,3.0,"Shahbaaz, Mohd;Ahmad, Faizan;Imtaiyaz Hassan, Md",56039303200;56108326000;7401625549,60020458;60020458;60020458,17,1,repositoryvor,Green,ICMR,BIC/12(04)/2012,Indian Council of Medical Research
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29442718,2-s2.0-0022312055,None,Analysis of the level of knowledge on diet in ...,1985-01-01,120108634.0,Zakladu Szkolenia i Oswiaty Zdrowotnej Instytu...,Warsaw,Poland,2,"Kuskowska-Wolk, A.;Puncewicz, B.",6602097024;6603405065,120108634;120108634,0,0,None,None,None,undefined,None
29442719,2-s2.0-0022295156,None,Assessment of dietary intake in diabetics,1985-01-01,120108634.0,Zakladu Szkolenia i Oswiaty Zdrowotnej Instytu...,Warsaw,Poland,2,"Kuskowska-Wolk, A.;Ilinska-Debniak, K.",6602097024;6506557146,120108634;120108634,0,0,None,None,None,undefined,None
29442720,2-s2.0-0021857140,None,Contents of ascorbic acid and 2-ascorbichydros...,1985-01-01,60021858.0,Uniwersytet Mikołaja Kopernika w Toruniu,Torun,Poland,3,"Gniot-Szulzycka, J.;Korzycka, W.;Stefanska, J.",6701780216;6504020998;6603414444,60021858;60021858;60021858,0,0,None,None,None,undefined,None
29442721,2-s2.0-85142401310,10.1017/S0967199422000284,Supplementation of porcine in vitro maturation...,2022-12-01,60120852;60115175;60019444;60010818,JSR Genetics Ltd;Topigs Norsvin;Canterbury Chr...,Driffield;Helvoirt;Canterbury;Canterbury,United Kingdom;Netherlands;United Kingdom;Unit...,10.0,"Serrano Albal, María;Silvestri, Giuseppe;Kiazi...",57220602738;57196370255;57202201190;5722655303...,60010818;60010818;60010818;60019444;60115175;6...,1,1,publisherhybridgold,Hybrid Gold,None,undefined,None


In [4]:
query = 'SELECT t.* FROM "biotechnology_author" t '
# df_patstat=pd.read_sql(query, connection)
df_patstat_author=pd.read_pickle('data/df_patstat.pickle')
df_patstat_author=df_patstat_author[df_patstat_author['person_id'].notnull()]
df_patstat_author['person_id']=df_patstat_author['person_id'].astype(int).astype(str)
df_patstat_author['psn_id']=df_patstat_author['psn_id'].astype(int).astype(str)
df_patstat_author['appln_id']=df_patstat_author['appln_id'].astype(int).astype(str)
df_patstat_author

,appln_id,person_id,invt_seq_nr,psn_name,psn_id,psn_sector,person_address
8,15763360,16,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
11,15768652,16,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
15,15772733,16,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
24,15972133,16,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
26,15978103,16,0.0,MRC (MEDICAL RESEARCH COUNCIL),22672891,GOV NON-PROFIT,"20 Park Crescent,London W1N 4AL"
...,...,...,...,...,...,...,...
40251181,567558584,85453972,0.0,"SHANGYAOTANG BIG HEALTH INDUSTRY GROUP CO., LTD.",185453972,UNKNOWN,None
40251187,557454227,85454359,5.0,SHE KEPENG,185454359,UNKNOWN,None
40251190,564771023,85438757,1.0,JAMES O. DAVIS,185438757,UNKNOWN,None
40251192,571323564,85440714,1.0,Richard Gagliardo,185440714,UNKNOWN,None


In [5]:
# query = 'SELECT t.* FROM "biotechnology_basic_info" t '
# df_info=pd.read_sql(query, connection)
# df_info.to_pickle('data/df_info.pickle')
df_info=pd.read_pickle('data/df_info.pickle')
df_info['appln_id']=df_info['appln_id'].astype(int).astype(str)
df_info

,appln_id,appln_auth,appln_nr,appln_kind,appln_filing_date,appln_filing_year,appln_nr_original,ipr_type,receiving_office,internat_appln_id,...,earliest_publn_date,earliest_publn_year,earliest_pat_publn_id,granted,docdb_family_id,inpadoc_family_id,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors
0,72,EP,04799975,A,2004-11-01 00:00:00,2004,04799975,PI,,39982171,...,2006-08-23 00:00:00,2006,283233012,Y,36676151,72,8,0,2,2
1,79,EP,05023512,A,2005-10-27 00:00:00,2005,05023512,PI,,0,...,2006-05-10 00:00:00,2006,281372800,Y,35677469,79,4,2,1,1
2,88,EP,05715031,A,2005-02-27 00:00:00,2005,05715031,PI,,14915915,...,2006-11-08 00:00:00,2006,283072913,Y,37818140,88,16,66,1,1
3,116,EP,06706886,A,2006-02-13 00:00:00,2006,06706886,PI,,17500685,...,2007-11-07 00:00:00,2007,280885333,Y,34933703,116,11,18,1,4
4,119,EP,06753961,A,2006-05-30 00:00:00,2006,06753961,PI,,17504472,...,2008-06-25 00:00:00,2008,279972414,Y,37000110,119,10,34,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10676891,575014548,CN,202121421775,U,2021-06-24 00:00:00,2021,202121421775,UM,,0,...,2022-07-01 00:00:00,2022,575014549,Y,82121976,575014548,1,0,0,0
10676892,575014578,CN,202120539293,U,2021-03-16 00:00:00,2021,202120539293,UM,,0,...,2022-07-01 00:00:00,2022,575014579,Y,75539902,548993159,6,9,1,6
10676893,575014590,CN,202121439309,U,2021-06-28 00:00:00,2021,202121439309,UM,,0,...,2022-07-01 00:00:00,2022,575014591,Y,82121995,575014590,1,0,0,0
10676894,575014598,CN,202122451292,U,2021-10-12 00:00:00,2021,202122451292,UM,,0,...,2022-07-01 00:00:00,2022,575014599,Y,82121999,575014598,1,0,0,0


In [6]:
# query = 'SELECT t.* FROM "biotechnology_citation" t '
# df_citation=pd.read_sql(query, connection)
# df_citation.to_pickle('data/biotechnology_citation.pickle')
df_citation=pd.read_pickle('data/biotechnology_citation.pickle')
df_citation['appln_id']=df_citation['appln_id'].astype(str)
df_citation['cited_appln_id']=df_citation['cited_appln_id'].astype(str)
df_citation

,appln_id,cited_appln_id,appln_filing_date,cited_appln_filing_date
0,58003622,51579672,2008-01-25 00:00:00,2000-03-16 00:00:00
1,530626640,50699641,2020-01-08 00:00:00,2006-04-28 00:00:00
2,437637376,53064399,2014-09-16 00:00:00,2007-03-28 00:00:00
3,407533390,4907984,2011-09-07 00:00:00,2005-10-19 00:00:00
4,336735820,50040999,2011-03-08 00:00:00,2003-02-04 00:00:00
...,...,...,...,...
66505530,52943861,51096435,2003-11-13 00:00:00,1995-06-05 00:00:00
66505531,484399662,449826595,2017-03-23 00:00:00,2013-10-04 00:00:00
66505532,15795899,11266514,2000-12-08 00:00:00,1993-07-28 00:00:00
66505533,527075716,450970980,2019-11-20 00:00:00,2015-12-24 00:00:00


In [7]:
df_citation['delta']=df_citation['appln_filing_date'] - df_citation['cited_appln_filing_date']
df_citation['delta']=df_citation['delta'].apply(lambda x:x.days/365)

In [8]:
df_citation[df_citation['cited_appln_id']=='6675034']

,appln_id,cited_appln_id,appln_filing_date,cited_appln_filing_date,delta
33376159,470494614,6675034,2016-05-20 00:00:00,2001-12-28 00:00:00,14.402740
33416349,416327266,6675034,2013-11-22 00:00:00,2001-12-28 00:00:00,11.909589
33632009,413720941,6675034,2013-09-23 00:00:00,2001-12-28 00:00:00,11.745205
33741635,423324732,6675034,2014-08-06 00:00:00,2001-12-28 00:00:00,12.613699


In [9]:
df_citation_5year=df_citation[df_citation['delta']<5].groupby(['cited_appln_id']).agg(citation_count_5year=('appln_id','count')).reset_index()
df_citation_5year

,cited_appln_id,citation_count_5year
0,10000128,3
1,10000173,1
2,10000208,1
3,10000250,1
4,10000270,3
...,...,...
3006933,9999322,1
3006934,99994,6
3006935,9999566,1
3006936,99996,9


In [10]:
df_match_result_with_address=pd.read_pickle('data/version/version_v1/df_all.pickle')
df_match_result_with_address.drop(columns=['invent_0_psn_name_encoding','invent_0_person_address_encoding',
                                           'person_address_encoding','affilname_encoding'],inplace=True)
df_match_result_with_address=df_match_result_with_address[
    (df_match_result_with_address['invent_0_psn_name_similarity'].apply(lambda x:compare_larger(x,0.6)))|
 (df_match_result_with_address['invent_0_person_address_similarity'].apply(lambda x:compare_larger(x,0.6)))|
    (df_match_result_with_address['person_address_similarity'].apply(lambda x:compare_larger(x,0.6)))
                                                         ]
df_match_result_with_address=df_match_result_with_address.explode(['author_name','afid',
                                                                   'affiliation_city',
                                                                   'affiliation_country',
                                                                   'affilname',
                                                                    'invent_0_psn_name_similarity',
                                                                   'invent_0_person_address_similarity',
                                                                   'person_address_similarity'])

df_match_result_with_address=df_match_result_with_address.drop_duplicates(['author_id','psn_id_matching','appln_id_matching','eid_matching'])
df_match_result_with_address=df_match_result_with_address[df_match_result_with_address['affilname'].notnull()]
df_match_result_with_address=df_match_result_with_address.reset_index(drop=True)
df_match_result_with_address

,labels,author_id,psn_id_matching,psn_id_matching_name,scopus_matching_name,appln_id_matching,eid_matching,doi_matching,title_scopus_matching,title_patstat_matching,...,psn_name,person_address,author_name,afid,affiliation_city,affiliation_country,affilname,invent_0_psn_name_similarity,invent_0_person_address_similarity,person_address_similarity
0,651,57632820300,36079383,Yamamoto; Itsuo,"Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,YAMAMOTO; ITSUO,"Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988
1,651,57632820300,36072517,"Yamamoto, Itsuo","Yamamoto, Itsuo",52568004,2-s2.0-0031763766,10.1080/028418698430458,"Relevance of a new impedance matching, or subt...",Apparatus for high-frequency hyperthermia,...,"YAMAMOTO, ITSUO","Yao-shi, Osaka","Yamamoto, Itsuo",100613492,Osaka,Japan,Yamamoto Vinita Co. Ltd.,0.648776,0.370988,0.370988
2,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.209711,0.149398
3,651,7005737279,27394586,"Robins, H., Ian","ROBINS, H. IAN",47242438,2-s2.0-0025534541,10.1007/978-1-4684-5766-7_17,Radiant heat systemic hyperthermia clinical tr...,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,...,"ROBINS, H., IAN",None,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.18224,0.18224
4,651,7005737279,27394585,"Robins, H. Ian","ROBINS, H. IAN",46482242,2-s2.0-0028322061,10.1016/0304-3835(94)90252-6,A new technological approach to radiant heat w...,Apparatus for inducing whole body hyperthermia...,...,"ROBINS, H. IAN",Madison,"Robins, H. I.",60016534,Madison,United States,UW Health,0.310088,0.209711,0.149398
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862936,3376,55746516000,4674685,Chen Lixue,"Chen, Lixue",526521667,2-s2.0-84914682049,10.1007/s10815-014-0351-8,Does IVF cleavage stage embryo quality affect ...,System for predicting continuous pregnancy of ...,...,CHEN LIXUE,None,"CHEN, Li Xue",60121975,Beijing,China,Peking University Third Hospital,0.697546,0.037507,0.037507
862937,3376,55746516000,4674685,Chen Lixue,"Chen, Lixue",526521667,2-s2.0-85120726748,10.1016/j.rbmo.2021.07.016,Maternal and neonatal outcomes following blast...,System for predicting continuous pregnancy of ...,...,CHEN LIXUE,None,"CHEN, Li Xue",60121975,Beijing,China,Peking University Third Hospital,0.697546,0.037507,0.037507
862938,3376,55746516000,4674685,Chen Lixue,"Chen, Lixue",526521667,2-s2.0-85089862443,10.1007/s43032-020-00247-x,Pregnancy Outcomes of Single/Double Blastocyst...,System for predicting continuous pregnancy of ...,...,CHEN LIXUE,None,"CHEN, Li Xue",60121975,Beijing,China,Peking University Third Hospital,0.697546,0.037507,0.037507
862939,3376,56256023400,24547578,Otsuki Junko,"Otsuki, Junko",524285915,2-s2.0-85021428936,10.1002/rmb2.12038,Retrospective comparative study of the factors...,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION,...,OTSUKI JUNKO,"c/o HANABUSA WOMEN'S CLINIC, 1-1-2, Sannomiya-...","Otsuki, Junko",60006108,Ibaraki,Japan,National Institute of Biomedical Innovation,0.345041,0.094858,0.218905


In [11]:
df_record=pd.read_pickle('data/df_record.pickle')
# del df_record['title_embedding']
df_record.loc[df_record['appln_id'].notnull(), 'appln_id'] = df_record.loc[df_record['appln_id'].notnull(), 'appln_id'].astype(int).astype(str)
df_record

,appln_id,title,eid,doi,source_id,title_embedding
0,6675034,Aquatic plant fertilizer and its production me...,NaN,NaN,NaN,"[-0.01938207, 0.09059494, -0.056675874, -0.108..."
1,15463575,AN ENZYME AND ENZYME PREPARATION WITH ENDOGLUC...,NaN,NaN,NaN,"[0.032466024, -0.039404966, -0.06192986, 0.051..."
2,17206714,SOIL AND WATER REMEDIATION AND ENHANCEMENT PRO...,NaN,NaN,NaN,"[-0.052692994, 0.018156238, 0.057793885, -0.03..."
3,27597733,DECOMPOSITION OF 2-NAPHTHALENESULFONIC ACID,NaN,NaN,NaN,"[-0.043281436, 0.047954354, -0.041820448, -0.0..."
4,39742737,PROCESS FOR DE-SIZING AND BLEACHING WOVEN FABR...,NaN,NaN,NaN,"[0.022980522, 0.085263304, -0.007640268, 0.003..."
...,...,...,...,...,...,...
29545075,NaN,Relative expression of the developmentally imp...,2-s2.0-85125811955,10.1017/S0967199421000976,19073.0,"[-0.0929608, -0.021178935, -0.072981216, -0.01..."
29545076,NaN,Advancements in mammalian X and y sperm differ...,2-s2.0-85125791972,10.1017/S0967199421000939,19073.0,"[-0.0072978716, 0.015270151, -0.031556655, -0...."
29545077,NaN,Effect of melatonin on the clinical outcome of...,2-s2.0-85125769983,10.1017/S0967199421000770,19073.0,"[0.026419124, 0.049893636, 0.032035816, 0.0320..."
29545078,NaN,Expression of genes associated with fertility ...,2-s2.0-85123940284,10.1017/S0967199421000745,19073.0,"[-0.010753157, -0.07279501, -0.026000984, 0.06..."


In [12]:
df_record[df_record['appln_id'].isin(list_tls901_filting_appln_id)]

,appln_id,title,eid,doi,source_id,title_embedding
0,6675034,Aquatic plant fertilizer and its production me...,NaN,NaN,NaN,"[-0.01938207, 0.09059494, -0.056675874, -0.108..."
1,15463575,AN ENZYME AND ENZYME PREPARATION WITH ENDOGLUC...,NaN,NaN,NaN,"[0.032466024, -0.039404966, -0.06192986, 0.051..."
2,17206714,SOIL AND WATER REMEDIATION AND ENHANCEMENT PRO...,NaN,NaN,NaN,"[-0.052692994, 0.018156238, 0.057793885, -0.03..."
3,27597733,DECOMPOSITION OF 2-NAPHTHALENESULFONIC ACID,NaN,NaN,NaN,"[-0.043281436, 0.047954354, -0.041820448, -0.0..."
4,39742737,PROCESS FOR DE-SIZING AND BLEACHING WOVEN FABR...,NaN,NaN,NaN,"[0.022980522, 0.085263304, -0.007640268, 0.003..."
...,...,...,...,...,...,...
6575532,573885104,Absorbable high-strength zinc alloy for medica...,NaN,NaN,NaN,"[-0.0770421, 0.101063825, 0.015055561, 0.00051..."
6575533,573885115,Tamper resistant catheter device,NaN,NaN,NaN,"[-0.055834655, 0.094606526, -0.013622771, -0.0..."
6575534,573885128,Wireless power receiver coil for implantable n...,NaN,NaN,NaN,"[-0.053963218, 0.07509954, -0.061199173, 0.011..."
6575535,573919112,METHOD FOR ISOLATING AND CULTURING CORD BLOOD ...,NaN,NaN,NaN,"[-0.044588502, -0.062603645, -0.09476462, -0.1..."


In [13]:
df_record[df_record['appln_id'].notnull()]

,appln_id,title,eid,doi,source_id,title_embedding
0,6675034,Aquatic plant fertilizer and its production me...,NaN,NaN,NaN,"[-0.01938207, 0.09059494, -0.056675874, -0.108..."
1,15463575,AN ENZYME AND ENZYME PREPARATION WITH ENDOGLUC...,NaN,NaN,NaN,"[0.032466024, -0.039404966, -0.06192986, 0.051..."
2,17206714,SOIL AND WATER REMEDIATION AND ENHANCEMENT PRO...,NaN,NaN,NaN,"[-0.052692994, 0.018156238, 0.057793885, -0.03..."
3,27597733,DECOMPOSITION OF 2-NAPHTHALENESULFONIC ACID,NaN,NaN,NaN,"[-0.043281436, 0.047954354, -0.041820448, -0.0..."
4,39742737,PROCESS FOR DE-SIZING AND BLEACHING WOVEN FABR...,NaN,NaN,NaN,"[0.022980522, 0.085263304, -0.007640268, 0.003..."
...,...,...,...,...,...,...
6575532,573885104,Absorbable high-strength zinc alloy for medica...,NaN,NaN,NaN,"[-0.0770421, 0.101063825, 0.015055561, 0.00051..."
6575533,573885115,Tamper resistant catheter device,NaN,NaN,NaN,"[-0.055834655, 0.094606526, -0.013622771, -0.0..."
6575534,573885128,Wireless power receiver coil for implantable n...,NaN,NaN,NaN,"[-0.053963218, 0.07509954, -0.061199173, 0.011..."
6575535,573919112,METHOD FOR ISOLATING AND CULTURING CORD BLOOD ...,NaN,NaN,NaN,"[-0.044588502, -0.062603645, -0.09476462, -0.1..."


In [14]:
df_record.columns

Index(['appln_id', 'title', 'eid', 'doi', 'source_id', 'title_embedding'], dtype='object')

In [15]:
with open('labels.pickle', 'rb') as f:
    labels=pickle.load(f)
    
# with open('clusterable_embedding.pickle', 'rb') as f:
#     clusterable_embedding=pickle.load(f)

In [16]:
df_record['labels']=labels
# df_record['clusterable_embedding']=list(clusterable_embedding)

In [17]:
df_record=df_record.merge(df_info,how='left',left_on='appln_id',right_on='appln_id')
df_record

,appln_id,title,eid,doi,source_id,title_embedding,labels,appln_auth,appln_nr,appln_kind,...,earliest_publn_date,earliest_publn_year,earliest_pat_publn_id,granted,docdb_family_id,inpadoc_family_id,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors
0,6675034,Aquatic plant fertilizer and its production me...,NaN,NaN,NaN,"[-0.01938207, 0.09059494, -0.056675874, -0.108...",-1,CN,01139285,A,...,2003-07-09 00:00:00,2003.0,286547315.0,N,4675207.0,6675034.0,1.0,4.0,1.0,1.0
1,15463575,AN ENZYME AND ENZYME PREPARATION WITH ENDOGLUC...,NaN,NaN,NaN,"[0.032466024, -0.039404966, -0.06192986, 0.051...",-1,WO,9500400,W,...,1996-04-18 00:00:00,1996.0,299028046.0,Y,26065315.0,404293.0,9.0,435.0,7.0,6.0
2,17206714,SOIL AND WATER REMEDIATION AND ENHANCEMENT PRO...,NaN,NaN,NaN,"[-0.052692994, 0.018156238, 0.057793885, -0.03...",-1,EP,96929826,A,...,1998-11-11 00:00:00,1998.0,297713822.0,Y,24074238.0,1325092.0,17.0,16.0,1.0,1.0
3,27597733,DECOMPOSITION OF 2-NAPHTHALENESULFONIC ACID,NaN,NaN,NaN,"[-0.043281436, 0.047954354, -0.041820448, -0.0...",-1,JP,4961991,A,...,1994-02-22 00:00:00,1994.0,301349790.0,N,12836252.0,27597733.0,1.0,0.0,2.0,3.0
4,39742737,PROCESS FOR DE-SIZING AND BLEACHING WOVEN FABR...,NaN,NaN,NaN,"[0.022980522, 0.085263304, -0.007640268, 0.003...",-1,KR,810004851,A,...,1983-11-09 00:00:00,1983.0,444841751.0,Y,9249673.0,1124436.0,14.0,13.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29545075,NaN,Relative expression of the developmentally imp...,2-s2.0-85125811955,10.1017/S0967199421000976,19073.0,"[-0.0929608, -0.021178935, -0.072981216, -0.01...",3356,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29545076,NaN,Advancements in mammalian X and y sperm differ...,2-s2.0-85125791972,10.1017/S0967199421000939,19073.0,"[-0.0072978716, 0.015270151, -0.031556655, -0....",-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29545077,NaN,Effect of melatonin on the clinical outcome of...,2-s2.0-85125769983,10.1017/S0967199421000770,19073.0,"[0.026419124, 0.049893636, 0.032035816, 0.0320...",-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29545078,NaN,Expression of genes associated with fertility ...,2-s2.0-85123940284,10.1017/S0967199421000745,19073.0,"[-0.010753157, -0.07279501, -0.026000984, 0.06...",-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# 添加引用
df_record=df_record.merge(df_scopus[['eid','citedby_count']],how='left',left_on='eid',right_on='eid')
df_record=df_record.merge(df_citation_5year,how='left',left_on='appln_id',right_on='cited_appln_id')


In [19]:
# df_record=df_record[df_record['labels']>100].reset_index()
df_record.loc[(df_record['appln_id'].notnull())&(df_record['citation_count_5year'].isnull()),'citation_count_5year']=0 
df_record.loc[(df_record['eid'].notnull())&(df_record['citedby_count'].isnull()),'citedby_count']=0 
df_record

,appln_id,title,eid,doi,source_id,title_embedding,labels,appln_auth,appln_nr,appln_kind,...,granted,docdb_family_id,inpadoc_family_id,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors,citedby_count,cited_appln_id,citation_count_5year
0,6675034,Aquatic plant fertilizer and its production me...,NaN,NaN,NaN,"[-0.01938207, 0.09059494, -0.056675874, -0.108...",-1,CN,01139285,A,...,N,4675207.0,6675034.0,1.0,4.0,1.0,1.0,NaN,NaN,0.0
1,15463575,AN ENZYME AND ENZYME PREPARATION WITH ENDOGLUC...,NaN,NaN,NaN,"[0.032466024, -0.039404966, -0.06192986, 0.051...",-1,WO,9500400,W,...,Y,26065315.0,404293.0,9.0,435.0,7.0,6.0,NaN,15463575,37.0
2,17206714,SOIL AND WATER REMEDIATION AND ENHANCEMENT PRO...,NaN,NaN,NaN,"[-0.052692994, 0.018156238, 0.057793885, -0.03...",-1,EP,96929826,A,...,Y,24074238.0,1325092.0,17.0,16.0,1.0,1.0,NaN,NaN,0.0
3,27597733,DECOMPOSITION OF 2-NAPHTHALENESULFONIC ACID,NaN,NaN,NaN,"[-0.043281436, 0.047954354, -0.041820448, -0.0...",-1,JP,4961991,A,...,N,12836252.0,27597733.0,1.0,0.0,2.0,3.0,NaN,NaN,0.0
4,39742737,PROCESS FOR DE-SIZING AND BLEACHING WOVEN FABR...,NaN,NaN,NaN,"[0.022980522, 0.085263304, -0.007640268, 0.003...",-1,KR,810004851,A,...,Y,9249673.0,1124436.0,14.0,13.0,1.0,1.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29545085,NaN,Relative expression of the developmentally imp...,2-s2.0-85125811955,10.1017/S0967199421000976,19073.0,"[-0.0929608, -0.021178935, -0.072981216, -0.01...",3356,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
29545086,NaN,Advancements in mammalian X and y sperm differ...,2-s2.0-85125791972,10.1017/S0967199421000939,19073.0,"[-0.0072978716, 0.015270151, -0.031556655, -0....",-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN
29545087,NaN,Effect of melatonin on the clinical outcome of...,2-s2.0-85125769983,10.1017/S0967199421000770,19073.0,"[0.026419124, 0.049893636, 0.032035816, 0.0320...",-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN
29545088,NaN,Expression of genes associated with fertility ...,2-s2.0-85123940284,10.1017/S0967199421000745,19073.0,"[-0.010753157, -0.07279501, -0.026000984, 0.06...",-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN


In [20]:
df_match_result_with_address.columns

Index(['labels', 'author_id', 'psn_id_matching', 'psn_id_matching_name',
       'scopus_matching_name', 'appln_id_matching', 'eid_matching',
       'doi_matching', 'title_scopus_matching', 'title_patstat_matching',
       'score_matching', 'invent_0_appln_id', 'invent_0_person_id',
       'invent_0_invt_seq_nr', 'invent_0_psn_id', 'invent_0_psn_name',
       'invent_0_person_address', 'appln_id', 'person_id', 'invt_seq_nr',
       'psn_id', 'psn_name', 'person_address', 'author_name', 'afid',
       'affiliation_city', 'affiliation_country', 'affilname',
       'invent_0_psn_name_similarity', 'invent_0_person_address_similarity',
       'person_address_similarity'],
      dtype='object')

In [21]:
df_record=df_record.merge(df_scopus[[ 'eid','coverDate', 'afid', 'affilname', 'affiliation_city', 'affiliation_country', 'author_count', 'author_names', 
            'author_ids', 'author_afids',   'openaccess', 'freetoread', 'freetoreadLabel', 'fund_acr', 'fund_no', 'fund_sponsor']],
                         how='left',left_on='eid',right_on='eid')

In [22]:
t3=pd.read_pickle('data/t3.pickle')
t3=t3.reset_index(level=1)
t3.columns=['author_idx','author_id','author_name']
t3=t3.merge(df_scopus[['eid']],left_index=True,right_index=True,how='left')
t3

,author_idx,author_id,author_name,eid
0,0,55479937900,"Kamthania, Mohit",2-s2.0-84991318817
0,1,57206773481,"Sharma, D. K.",2-s2.0-84991318817
1,0,57197638841,"Ghosh, Tuhin",2-s2.0-84982787473
1,1,7402167211,"Basu, Ankita",2-s2.0-84982787473
1,2,57213724746,"Adhikari, Dipan",2-s2.0-84982787473
...,...,...,...,...
29442721,9,55845463100,"Griffin, Darren K.",2-s2.0-85142401310
29442722,0,57917416100,"Hashimoto, Shizuya",2-s2.0-85133137634
29442722,1,57917450100,"Kinjo, Keita",2-s2.0-85133137634
29442722,2,55689793000,"Saito, Takako",2-s2.0-85133137634


# 形成完整数据集

In [23]:
# filiting_appln=df_record[(df_record['appln_filing_year']<2016)&(df_record['appln_filing_year']>=1980)]['appln_id'].unique()
# filiting_appln

In [24]:
df_paper=df_match_result_with_address[['author_id']].drop_duplicates().merge(t3[[ 'author_id', 'author_name', 'eid']],how='left',left_on='author_id',right_on='author_id'
                                                                    ).merge(df_record[[ 'title', 'eid', 'doi', 'source_id',
       'labels', 'coverDate', 'afid', 'affilname', 'affiliation_city', 'affiliation_country', 'author_count', 'author_names', 
        'author_ids', 'author_afids', 'openaccess',  'freetoread', 'freetoreadLabel', 'fund_acr', 'fund_no', 'fund_sponsor','citedby_count']], 
                                                                            how='inner',left_on='eid',right_on='eid')

df_paper['coverDate_year']=df_paper['coverDate'].apply(lambda x:x.year)
df_paper

,author_id,author_name,eid,title,doi,source_id,labels,coverDate,afid,affilname,...,author_ids,author_afids,openaccess,freetoread,freetoreadLabel,fund_acr,fund_no,fund_sponsor,citedby_count,coverDate_year
0,57632820300,"Yamamoto, Itsuo",2-s2.0-0031763766,"Relevance of a new impedance matching, or subt...",10.1080/028418698430458,28668.0,651,1998-11-17,60026645;100613492,Kansai Medical University;Yamamoto Vinita Co. ...,...,35398882900;7102392345;7005295371;10340711700;...,60026645;60026645;60026645;60026645;60026645;6...,0,None,None,None,undefined,None,1.0,1998
1,57632820300,"Yamamoto, I.",2-s2.0-0029100367,Newly developed transurethral radiofrequency t...,10.3109/02656739509022495,29901.0,101,1995-01-01,60011001;60007498;101815157,Kyoto University;Kurashiki Central Hospital;Ya...,...,7006262975;35397858000;57632820300;7202264427;...,60011001;60007498;101815157;60011001;60011001;...,0,None,None,None,undefined,None,6.0,1995
2,56581333900,"Yoshida, O.",2-s2.0-0029100367,Newly developed transurethral radiofrequency t...,10.3109/02656739509022495,29901.0,101,1995-01-01,60011001;60007498;101815157,Kyoto University;Kurashiki Central Hospital;Ya...,...,7006262975;35397858000;57632820300;7202264427;...,60011001;60007498;101815157;60011001;60011001;...,0,None,None,None,undefined,None,6.0,1995
3,57632820300,"Yamamoto, Itsuo",2-s2.0-0032877310,Increased resistance of the radiosensitive M10...,10.2307/3580332,29515.0,2145,1999-09-01,60026645;60017563;60013473;118143905;100613492,Kansai Medical University;Weizmann Institute o...,...,7005295371;7501852661;35495736900;7402799384;5...,60026645-118143905;60013473-60017563;60013473;...,0,None,None,None,undefined,None,8.0,1999
4,7005737279,"Robins, H. I.",2-s2.0-0036162128,Pineal sarcoma,10.1007/s701-002-8277-6,20412.0,2918,2002-02-12,60017638;60016534;106655991,Logan Health Medical Center;UW Health;St Luke'...,...,57191690802;6701347661;7004617722;7203024131;7...,60016534;60016534;60016534;60016534;60016534;1...,0,None,None,None,undefined,None,5.0,2002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798115,56256023400,"Otsuki, Junko",2-s2.0-85081307887,Predictive factors influencing pregnancy rate ...,10.1002/rmb2.12322,130083.0,3376,2020-04-01,60023147;101983163,Okayama University;Hanabusa Women's Clinic,...,57215607080;36447286200;57191646754;5625602340...,101983163;101983163;101983163;101983163-600231...,1,repositoryvor,Green,None,undefined,None,5.0,2020
4798116,56256023400,"Otsuki, Junko",2-s2.0-85115930058,Clinical outcome of intrauterine infusion of p...,10.1002/rmb2.12417,130083.0,-1,2022-01-01,60023147;101983163,Okayama University;Hanabusa Women's Clinic,...,57202938659;36447286200;55185568600;5625602340...,101983163;101983163;101983163;101983163-600231...,1,repositoryvor,Green,None,undefined,None,3.0,2022
4798117,56256023400,"Otsuki, Junko",2-s2.0-85145038597,Are tri-pronuclear embryos that show two norma...,10.1002/rmb2.12462,130083.0,-1,2022-01-01,60023147;108171410,Okayama University;Okayama Couple’s Clinic,...,57211503322;57209114324;56256023400;6603956373...,108171410;108171410;60023147;108171410;108171410,1,repositoryvor,Green,None,undefined,None,1.0,2022
4798118,56256023400,"Otsuki, Junko",2-s2.0-85143330794,A novel system based on artificial intelligenc...,10.1002/rmb2.12443,130083.0,-1,2022-01-01,60023147;129022284;101983163,Okayama University;NextGeM inc.;Hanabusa Women...,...,36447286200;57658857600;58034988300;5803501620...,101983163;129022284;129022284;101983163;101983...,1,repositoryvor,Green,None,undefined,None,3.0,2022


In [25]:
df_patent=df_match_result_with_address[['person_id']].drop_duplicates().merge(df_patstat_author[['person_id','appln_id']], how='left',left_on='person_id',right_on='person_id').merge(
    df_record[['appln_id', 'title', 'labels','appln_auth', 'appln_nr', 'appln_kind', 'appln_filing_date', 'appln_filing_year', 'appln_nr_original', 'ipr_type', 'receiving_office', 'internat_appln_id', 'int_phase', 'reg_phase',
       'nat_phase', 'earliest_filing_date', 'earliest_filing_year', 'earliest_filing_id', 'earliest_publn_date', 'earliest_publn_year',
       'earliest_pat_publn_id', 'granted', 'docdb_family_id', 'inpadoc_family_id', 'docdb_family_size', 'nb_citing_docdb_fam',
       'nb_applicants', 'nb_inventors', 'citedby_count', 'cited_appln_id', 'citation_count_5year']],
                                                                              how='inner',left_on='appln_id',right_on='appln_id')
df_patent

,person_id,appln_id,title,labels,appln_auth,appln_nr,appln_kind,appln_filing_date,appln_filing_year,appln_nr_original,...,granted,docdb_family_id,inpadoc_family_id,docdb_family_size,nb_citing_docdb_fam,nb_applicants,nb_inventors,citedby_count,cited_appln_id,citation_count_5year
0,9384898,52568004,Apparatus for high-frequency hyperthermia,651,US,66035184,A,1984-10-12 00:00:00,1984.0,660351,...,Y,16506615.0,34334329.0,2.0,8.0,2.0,2.0,NaN,52568004,3.0
1,9384900,52568004,Apparatus for high-frequency hyperthermia,651,US,66035184,A,1984-10-12 00:00:00,1984.0,660351,...,Y,16506615.0,34334329.0,2.0,8.0,2.0,2.0,NaN,52568004,3.0
2,5762813,46482242,Apparatus for inducing whole body hyperthermia...,651,US,5388393,A,1993-04-27 00:00:00,1993.0,053883,...,Y,21987202.0,1290643.0,10.0,40.0,1.0,2.0,NaN,46482242,2.0
3,5762812,46482242,Apparatus for inducing whole body hyperthermia...,651,US,5388393,A,1993-04-27 00:00:00,1993.0,053883,...,Y,21987202.0,1290643.0,10.0,40.0,1.0,2.0,NaN,46482242,2.0
4,37692963,47242438,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,651,WO,9404593,W,1994-04-26 00:00:00,1994.0,US9404593,...,Y,21987202.0,1290643.0,10.0,40.0,1.0,2.0,NaN,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
737742,62535779,520316723,METHODS FOR DETERMINING QUALITY OF EMBRYO,3376,JP,2019095944,A,2019-05-22 00:00:00,2019.0,2019095944,...,N,48698981.0,419101526.0,12.0,9.0,4.0,4.0,NaN,NaN,0.0
737743,62535779,567010260,METHODS FOR DETERMINING QUALITY OF EMBRYO,3376,JP,2021169295,A,2021-10-15 00:00:00,2021.0,2021169295,...,N,48698981.0,419101526.0,12.0,9.0,4.0,4.0,NaN,NaN,0.0
737744,55334074,564766225,PREDICTING EMBRYO IMPLANTATION PROBABILITY,3376,WO,2021056255,W,2021-07-12 00:00:00,2021.0,IB2021/056255,...,N,79172761.0,563645794.0,2.0,0.0,1.0,4.0,NaN,NaN,0.0
737745,50190939,564766225,PREDICTING EMBRYO IMPLANTATION PROBABILITY,3376,WO,2021056255,W,2021-07-12 00:00:00,2021.0,IB2021/056255,...,N,79172761.0,563645794.0,2.0,0.0,1.0,4.0,NaN,NaN,0.0


In [26]:
len(df_patent['person_id'].unique())

119405

In [27]:
df_paper[['eid']].drop_duplicates()

,eid
0,2-s2.0-0031763766
1,2-s2.0-0029100367
3,2-s2.0-0032877310
4,2-s2.0-0036162128
5,2-s2.0-0025565999
...,...
4798115,2-s2.0-85081307887
4798116,2-s2.0-85115930058
4798117,2-s2.0-85145038597
4798118,2-s2.0-85143330794


In [28]:
df_paper[['eid']].drop_duplicates().to_csv('version_4/v4_df_paper_eid.csv',encoding='utf-8-sig')
df_patent[['appln_id']].drop_duplicates().to_csv('version_4/v4_df_patent_appln_id.csv',encoding='utf-8-sig')

In [29]:
df_paper_eid_abs = pd.read_sql(sql='SELECT t.* FROM "v4_df_paper_eid_abs" t', con=connection)
df_paper_eid_abs=df_paper_eid_abs.drop_duplicates('eid')
df_paper_eid_abs

/tmp/ipykernel_13444/2402710028.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_paper_eid_abs = pd.read_sql(sql='SELECT t.* FROM "v4_df_paper_eid_abs" t', con=connection)


,eid,description
0,2-s2.0-85009896689,The best way to protect yield loss of soybean ...
1,2-s2.0-85008864359,Degradation of residual lignin in kraft pulp b...
2,2-s2.0-85015302975,A Gram-positive bacterium was isolated from ma...
3,2-s2.0-85017423684,"A MYB transcription factor gene, RsMYB1, from ..."
4,2-s2.0-85017416810,Pretreatment of miscanthus is essential for ef...
...,...,...
3270366,2-s2.0-85125791972,Mammal sex determination depends on whether th...
3270367,2-s2.0-85125769983,To explore whether embryo culture with melaton...
3270368,2-s2.0-85129295753,Summary Our previous studies have already reve...
3270369,2-s2.0-85128146377,Summary Embryo quality determines the success ...


In [30]:
df_patent_appln_id_abs = pd.read_sql(sql='SELECT t.* FROM "v4_df_patent_appln_id_abs" t', con=connection)
df_patent_appln_id_abs=df_patent_appln_id_abs.drop_duplicates('appln_id')
df_patent_appln_id_abs['appln_id']=df_patent_appln_id_abs['appln_id'].astype(str)
df_patent_appln_id_abs

/tmp/ipykernel_13444/2823665951.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_patent_appln_id_abs = pd.read_sql(sql='SELECT t.* FROM "v4_df_patent_appln_id_abs" t', con=connection)


,appln_id,appln_abstract
0,4592011,The present invention relates to a polypeptide...
1,15567268,1. An isolated sequence of the DNA molecule wh...
2,17060288,Lactic acid bacteria of the genus Lactobacillu...
3,17197663,"Methods and compositions for the prevention, t..."
4,29350367,PURPOSE:To obtain a new antibiotic substance P...
...,...,...
476896,573880728,The invention concerns treating Tauopathies su...
476897,573923811,An embodiment of the present invention provide...
476898,573918875,The present invention relates to a sleep apnea...
476899,573919037,Provided is a variant of a spike protein that ...


In [31]:
df_patent_appln_id_abs['appln_abstract_embedding']=list(model.encode(df_patent_appln_id_abs['appln_abstract'].astype(str).tolist(),show_progress_bar=True,convert_to_numpy=True))


Batches:   0%|          | 0/14904 [00:00<?, ?it/s]

In [32]:
df_paper_eid_abs['description_embedding']=list(model.encode(df_paper_eid_abs['description'].astype(str).tolist(),show_progress_bar=True,convert_to_numpy=True))


Batches:   0%|          | 0/102200 [00:00<?, ?it/s]

In [33]:
df_ppp=df_match_result_with_address[['author_id','person_id']].drop_duplicates()
df_ppp=df_ppp.merge(df_paper[['author_id', 'eid', 'title', 'doi', 'source_id']],how='inner',left_on='author_id',right_on='author_id')
df_ppp=df_ppp.merge(df_patent[['person_id','appln_id', 'title']],how='inner',left_on='person_id',right_on='person_id')
df_ppp=df_ppp.rename({'title_x':'title_paper','title_y':'title_patent'},axis=1)
df_ppp

,author_id,person_id,eid,title_paper,doi,source_id,appln_id,title_patent
0,57632820300,9384898,2-s2.0-0031763766,"Relevance of a new impedance matching, or subt...",10.1080/028418698430458,28668.0,52568004,Apparatus for high-frequency hyperthermia
1,57632820300,9384898,2-s2.0-0029100367,Newly developed transurethral radiofrequency t...,10.3109/02656739509022495,29901.0,52568004,Apparatus for high-frequency hyperthermia
2,57632820300,9384898,2-s2.0-0032877310,Increased resistance of the radiosensitive M10...,10.2307/3580332,29515.0,52568004,Apparatus for high-frequency hyperthermia
3,57632820300,9384900,2-s2.0-0031763766,"Relevance of a new impedance matching, or subt...",10.1080/028418698430458,28668.0,52568004,Apparatus for high-frequency hyperthermia
4,57632820300,9384900,2-s2.0-0029100367,Newly developed transurethral radiofrequency t...,10.3109/02656739509022495,29901.0,52568004,Apparatus for high-frequency hyperthermia
...,...,...,...,...,...,...,...,...
147670510,56256023400,79414020,2-s2.0-85081307887,Predictive factors influencing pregnancy rate ...,10.1002/rmb2.12322,130083.0,524285915,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION
147670511,56256023400,79414020,2-s2.0-85115930058,Clinical outcome of intrauterine infusion of p...,10.1002/rmb2.12417,130083.0,524285915,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION
147670512,56256023400,79414020,2-s2.0-85145038597,Are tri-pronuclear embryos that show two norma...,10.1002/rmb2.12462,130083.0,524285915,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION
147670513,56256023400,79414020,2-s2.0-85143330794,A novel system based on artificial intelligenc...,10.1002/rmb2.12443,130083.0,524285915,ANALYSIS SOFTWARE AND DEVICE FOR EMBRYO SELECTION


In [34]:
print('df_ppp author_id number--',len(df_ppp['author_id'].unique()))
print('df_ppp person_id number--',len(df_ppp['person_id'].unique()))
print('df_ppp appln_id number--',len(df_ppp['appln_id'].unique()))
print('df_ppp eid number--',len(df_ppp['eid'].unique()))

print('df_patent appln_id number--',len(df_patent['appln_id'].unique()))
print('df_paper eid number--',len(df_paper['eid'].unique()))
print('paper-patent pair number--',len(df_ppp.drop_duplicates(['appln_id','eid'])))


df_ppp author_id number-- 106150
df_ppp person_id number-- 119405
df_ppp appln_id number-- 476901
df_ppp eid number-- 3270371
df_patent appln_id number-- 476901
df_paper eid number-- 3270371
paper-patent pair number-- 145582942


In [35]:
# df_ppp.to_pickle('version_4/df_ppp.pickle')
# df_paper.to_pickle('version_4/df_paper.pickle')
df_patent.to_pickle('version_4/df_patent.pickle')


In [7]:
df_paper=pd.read_pickle('version_4/df_paper.pickle')
df_patent=pd.read_pickle('version_4/df_patent.pickle')

In [8]:
df_paper['title_embedding']=list(model.encode(df_paper['title'].astype(str).tolist(),show_progress_bar=True,convert_to_numpy=True))
df_patent['title_embedding']=list(model.encode(df_patent['title'].astype(str).tolist(),show_progress_bar=True,convert_to_numpy=True))


Batches:   0%|          | 0/149942 [00:00<?, ?it/s]

Batches:   0%|          | 0/23055 [00:00<?, ?it/s]

In [24]:
dict_eid_title={eid:title_embedding for eid,title_embedding in zip(df_paper['eid'],df_paper['title_embedding'])}
dict_appln_title={appln:title_embedding for appln,title_embedding in zip(df_patent['appln_id'],df_patent['title_embedding'])}
dict_eid_abs={eid:description_embedding for eid,description_embedding in zip(df_paper_eid_abs['eid'],df_paper_eid_abs['description'])}
dict_eid_abs_embed={eid:description_embedding for eid,description_embedding in zip(df_paper_eid_abs['eid'],df_paper_eid_abs['description_embedding'])}
dict_appln_abs={appln:appln_abstract_embedding for appln,appln_abstract_embedding in zip(df_patent_appln_id_abs['appln_id'],df_patent_appln_id_abs['appln_abstract'])}
dict_appln_abs_embed={appln:appln_abstract_embedding for appln,appln_abstract_embedding in zip(df_patent_appln_id_abs['appln_id'],df_patent_appln_id_abs['appln_abstract_embedding'])}


In [25]:
with open('dict_eid_title.pickle', 'wb') as f:
    pickle.dump(dict_eid_title,f)
    
with open('dict_appln_title.pickle', 'wb') as f:
    pickle.dump(dict_appln_title,f)
    
with open('dict_eid_abs.pickle', 'wb') as f:
    pickle.dump(dict_eid_abs,f)
    
with open('dict_appln_abs.pickle', 'wb') as f:
    pickle.dump(dict_appln_abs,f)
    
with open('dict_eid_abs_embed.pickle', 'wb') as f:
    pickle.dump(dict_eid_abs_embed,f)
    
with open('dict_appln_abs_embed.pickle', 'wb') as f:
    pickle.dump(dict_appln_abs_embed,f)

In [ ]:
# df_ppp=pd.read_pickle('version_3/df_ppp.pickle')
# df_paper=pd.read_pickle('version_3/df_paper.pickle')
# df_patent=pd.read_pickle('version_3/df_patent.pickle')


In [73]:
chunk_size = 15000000

# 获取原始DataFrame的行数
total_rows = len(df_ppp)

# 计算要拆分的小DataFrame的数量
num_chunks = (total_rows // chunk_size) + 1

# 拆分DataFrame为小DataFrame列表
small_dfs = [df_ppp.iloc[i * chunk_size:(i + 1) * chunk_size] for i in range(num_chunks)]


In [112]:
cnt=0
for small_df in small_dfs:
    cnt+=1
    small_df.to_pickle('version_4/df_ppp_ori_{}.pickle'.format(str(cnt)))

In [ ]:
df_paper=pd.read_pickle('version_4/df_paper.pickle')
df_patent=pd.read_pickle('version_4/df_patent.pickle')

# 生成GitHub

In [2]:
import pandas as pd

query = 'SELECT t."appln_id", t."appln_kind", t."appln_auth", t."appln_nr_original" FROM "biotechnology_basic_info" t'
df_basic_info=pd.read_sql(query, connection)
# df_scopus=pd.read_pickle('data/df_scopus.pickle')
df_basic_info['appln_id']=df_basic_info['appln_id'].astype(str)

/tmp/ipykernel_88032/1675333474.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_basic_info=pd.read_sql(query, connection)


In [81]:
df_paper_github=df_paper[['author_name','title', 'doi']]
df_paper_github.to_csv('new_github/paper.csv',encoding='utf-8-sig',index=None)
df_paper_github

,author_name,title,doi
0,"Yamamoto, Itsuo","Relevance of a new impedance matching, or subt...",10.1080/028418698430458
1,"Yamamoto, I.",Newly developed transurethral radiofrequency t...,10.3109/02656739509022495
2,"Yoshida, O.",Newly developed transurethral radiofrequency t...,10.3109/02656739509022495
3,"Yamamoto, Itsuo",Increased resistance of the radiosensitive M10...,10.2307/3580332
4,"Robins, H. I.",Pineal sarcoma,10.1007/s701-002-8277-6
...,...,...,...
4798115,"Otsuki, Junko",Predictive factors influencing pregnancy rate ...,10.1002/rmb2.12322
4798116,"Otsuki, Junko",Clinical outcome of intrauterine infusion of p...,10.1002/rmb2.12417
4798117,"Otsuki, Junko",Are tri-pronuclear embryos that show two norma...,10.1002/rmb2.12462
4798118,"Otsuki, Junko",A novel system based on artificial intelligenc...,10.1002/rmb2.12443


In [38]:
df_paper.columns

Index(['author_id', 'author_name', 'eid', 'title', 'doi', 'source_id',
       'labels', 'coverDate', 'afid', 'affilname', 'affiliation_city',
       'affiliation_country', 'author_count', 'author_names', 'author_ids',
       'author_afids', 'openaccess', 'freetoread', 'freetoreadLabel',
       'fund_acr', 'fund_no', 'fund_sponsor', 'citedby_count',
       'coverDate_year'],
      dtype='object')

In [82]:
df_paper_github.sample(n=10000,random_state=42).to_csv('new_github/paper-sample.csv',encoding='utf-8-sig',index=None)

In [95]:
dict_psn_name={person_id:psn_name for person_id,psn_name in zip(df_patstat_author['person_id'],df_patstat_author['person_id'])}

In [97]:
df_patent.columns

Index(['person_id', 'appln_id', 'title', 'appln_auth', 'appln_nr',
       'appln_kind', 'appln_filing_date', 'appln_filing_year',
       'appln_nr_original', 'ipr_type', 'receiving_office',
       'internat_appln_id', 'int_phase', 'reg_phase', 'nat_phase',
       'earliest_filing_date', 'earliest_filing_year', 'earliest_filing_id',
       'earliest_publn_date', 'earliest_publn_year', 'earliest_pat_publn_id',
       'granted', 'docdb_family_id', 'inpadoc_family_id', 'docdb_family_size',
       'nb_citing_docdb_fam', 'nb_applicants', 'nb_inventors', 'citedby_count',
       'cited_appln_id', 'citation_count_5year', 'psn_name'],
      dtype='object')

In [99]:
df_patent['psn_name']=df_patent['person_id'].map(dict_psn_name)
df_patent_github=df_patent[['title','appln_auth', 'appln_nr','appln_kind', 'appln_filing_date', 'appln_filing_year', 'appln_nr_original', 'ipr_type']]
df_patent_github.to_csv('new_github/patent.csv',encoding='utf-8-sig',index=None)
df_patent_github

,title,appln_auth,appln_nr,appln_kind,appln_filing_date,appln_filing_year,appln_nr_original,ipr_type
0,Apparatus for high-frequency hyperthermia,US,66035184,A,1984-10-12 00:00:00,1984.0,660351,PI
1,Apparatus for high-frequency hyperthermia,US,66035184,A,1984-10-12 00:00:00,1984.0,660351,PI
2,Apparatus for inducing whole body hyperthermia...,US,5388393,A,1993-04-27 00:00:00,1993.0,053883,PI
3,Apparatus for inducing whole body hyperthermia...,US,5388393,A,1993-04-27 00:00:00,1993.0,053883,PI
4,METHOD AND APPARATUS FOR HYPERTHERMIA TREATMENT,WO,9404593,W,1994-04-26 00:00:00,1994.0,US9404593,PI
...,...,...,...,...,...,...,...,...
737742,METHODS FOR DETERMINING QUALITY OF EMBRYO,JP,2019095944,A,2019-05-22 00:00:00,2019.0,2019095944,PI
737743,METHODS FOR DETERMINING QUALITY OF EMBRYO,JP,2021169295,A,2021-10-15 00:00:00,2021.0,2021169295,PI
737744,PREDICTING EMBRYO IMPLANTATION PROBABILITY,WO,2021056255,W,2021-07-12 00:00:00,2021.0,IB2021/056255,PI
737745,PREDICTING EMBRYO IMPLANTATION PROBABILITY,WO,2021056255,W,2021-07-12 00:00:00,2021.0,IB2021/056255,PI


In [101]:
df_patent_github.sample(n=10000,random_state=42).to_csv('new_github/patent-sample.csv',encoding='utf-8-sig',index=None)

In [ ]:
df_ppp=pd.concat([pd.read_pickle('version_4/'+i) for i in os.listdir('version_4') if "df_ppp_part_" in i])
df_ppp

In [4]:
len(df_ppp['author_id'].unique())

105722

In [5]:
len(df_ppp['appln_id'].unique())

333835

In [6]:
len(df_ppp['eid'].unique())

2405006

In [10]:
df_ppp[['author_id', 'person_id', 'eid', 'title_paper', 'doi', 'source_id',
       'appln_id', 'title_patent', 'simility','simility_abstract', 'Word_overlap_score', 
        'Word_overlap_score_abs',  'y_pred']].to_pickle('version_4/df_ppp_pred.pickle')

In [11]:
df_ppp[[ 'doi', 'source_id','title_paper','title_patent', 'y_pred']].to_csv('new_github/patent-paper-pair.csv',encoding='utf-8-sig',index=None)

In [12]:
df_ppp[[ 'doi', 'source_id','title_paper','title_patent', 'y_pred']].sample(n=10000,random_state=42).to_csv('new_github/patent-paper-pair-sample.csv',encoding='utf-8-sig',index=None)

In [36]:
# # df_ppp_github.to_csv('new_github/patent-paper-pair.csv',encoding='utf-8-sig',index=None)
# df_ppp_github.sample(n=10000,random_state=42).to_csv('new_github/patent-paper-pair-sample.csv',encoding='utf-8-sig',index=None)
# df_ppp_github